In [1]:
from torchvision.models import detection
from torchvision.io import read_image,read_file
import numpy as np
import torch
import torchvision.transforms as transforms
import time


c:\Users\mujad\.conda\envs\pytorch\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import matplotlib.pyplot as plt
import torchvision.transforms.functional as F
plt.rcParams["savefig.bbox"] = "tight"


def plot(imgs, **imshow_kwargs):
    if not isinstance(imgs[0], list):
        # Make a 2d grid even if there's just 1 row
        imgs = [imgs]

    num_rows = len(imgs)
    num_cols = len(imgs[0])
    _, axs = plt.subplots(nrows=num_rows, ncols=num_cols, squeeze=False)
    for row_idx, row in enumerate(imgs):
        for col_idx, img in enumerate(row):
            ax = axs[row_idx, col_idx]
            img = F.to_pil_image(img.to("cpu"))
            ax.imshow(np.asarray(img), **imshow_kwargs)
            ax.set(xticklabels=[], yticklabels=[], xticks=[], yticks=[])

    plt.tight_layout()

In [3]:
classes=[]
with open("coco_classlabels.txt") as f:
    classes=[line.strip() for line in f.readlines()]

In [10]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
CLASSES = classes
COLORS = np.random.randint(0, 255, size=(len(CLASSES), 3))
MIN_CONFIDENCE= 0.90

In [14]:
COLORS = [tuple(color) for color in COLORS]

In [18]:
COLORS[0]

(192, 222, 40)

In [6]:
model = detection.fasterrcnn_resnet50_fpn(pretrained=True, progress=True,
	num_classes=len(CLASSES), pretrained_backbone=True).to(DEVICE)

model.eval();

c:\Users\mujad\.conda\envs\pytorch\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\mujad\.conda\envs\pytorch\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=FasterRCNN_ResNet50_FPN_Weights.COCO_V1`. You can also use `weights=FasterRCNN_ResNet50_FPN_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
c:\Users\mujad\.conda\envs\pytorch\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained_backbone' is deprecated since 0.13 and may be removed in the future, please use 'weights_backbone' instead.
  warnings.warn(
c:\Users\mujad\.conda\envs\pytorch\lib\site-packages\torchvision\models\_uti

In [7]:
from torchvision.io import read_video

frames, _, _ = read_video('./sample.mp4', output_format="TCHW")

c:\Users\mujad\.conda\envs\pytorch\lib\site-packages\torchvision\io\video.py:162: UserWarning: The pts_unit 'pts' gives wrong results. Please use pts_unit 'sec'.
  warnings.warn("The pts_unit 'pts' gives wrong results. Please use pts_unit 'sec'.")


In [12]:
from torchvision.utils import draw_bounding_boxes
new_frames = torch.zeros_like(frames)
frame_idx = 0
for frame in frames:
    transform = transforms.Compose([
        transforms.ToPILImage(),
        transforms.ToTensor()
    ])

    transformed_image = transform(frame)

    transformed_image = transformed_image.unsqueeze(0)

    transformed_image = transformed_image.to(DEVICE)
    detections = model(transformed_image)[0]

    idx = detections['scores'] > 0.90
    labels = [f'{CLASSES[x-1].capitalize()}: {detections["scores"][x]*100:.2f}%' for x in detections['labels'][idx]]

    box_image=draw_bounding_boxes(frame,detections['boxes'][idx],labels=labels,width=4,font_size=25,font='arial', colors=COLORS)

    new_frames[frame_idx] = box_image
    frame_idx += 1

c:\Users\mujad\.conda\envs\pytorch\lib\site-packages\torchvision\utils.py:210: UserWarning: boxes doesn't contain any box. No box was drawn
  warnings.warn("boxes doesn't contain any box. No box was drawn")


In [13]:
from torchvision.io import write_video

write_video('./output.mp4',new_frames.permute(0,2,3,1),fps=30)